In [2]:
import os
import cv2
import numpy as np
from PIL import Image

# Parámetros de normalización
TARGET_SIZE = (224, 224)  # Dimensiones objetivo
TARGET_BRIGHTNESS = 120  # Brillo promedio deseado
TARGET_LAPLACIAN_VARIANCE = 100  # Varianza deseada para el enfoque

# Función para ajustar el brillo
def adjust_brightness(image, target_brightness):
    """Ajustar el brillo promedio de la imagen al nivel deseado."""
    current_brightness = np.mean(image)
    brightness_ratio = target_brightness / current_brightness
    adjusted = np.clip(image * brightness_ratio, 0, 255).astype(np.uint8)
    return adjusted

# Función para ajustar el enfoque
def adjust_focus(image, target_variance):
    """Ajustar el enfoque (nitidez) mediante un filtro Laplaciano."""
    current_variance = cv2.Laplacian(image, cv2.CV_64F).var()
    if current_variance < target_variance:
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # Filtro de nitidez
        return cv2.filter2D(image, -1, kernel)
    return image

# Función principal de normalización
def normalize_image(image_path, output_path):
    try:
        # Cargar la imagen
        img = Image.open(image_path).convert('RGB')
        img_resized = img.resize(TARGET_SIZE)

        # Convertir a array NumPy
        img_array = np.array(img_resized)

        # Ajustar brillo
        img_brightness_adjusted = adjust_brightness(img_array, TARGET_BRIGHTNESS)

        # Ajustar enfoque
        img_final = adjust_focus(img_brightness_adjusted, TARGET_LAPLACIAN_VARIANCE)

        # Guardar imagen normalizada
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        cv2.imwrite(output_path, cv2.cvtColor(img_final, cv2.COLOR_RGB2BGR))
        print(f"Imagen normalizada y guardada: {output_path}")

    except Exception as e:
        print(f"Error al procesar {image_path}: {e}")

# Función para procesar carpetas dinámicamente
def process_directory(source_dir, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            src_path = os.path.join(source_dir, filename)
            dest_path = os.path.join(destination_dir, filename)
            normalize_image(src_path, dest_path)

# Rutas de carpetas
source_path_1 = "classified/no"
source_path_2 = "classified/yes"
source_path_3 = "classified/pastry"
destination_path_1 = "normalized/no"
destination_path_2 = "normalized/yes"
destination_path_3 = "normalized/pastry"

# Procesar imágenes
print(f"Normalizando imágenes desde {source_path_1}...")
process_directory(source_path_1, destination_path_1)
print(f"Normalizando imágenes desde {source_path_2}...")
process_directory(source_path_2, destination_path_2)
print(f"Normalizando imágenes desde {source_path_3}...")
process_directory(source_path_3, destination_path_3)

print("¡Normalización completa!")


ModuleNotFoundError: No module named 'cv2'